# Pipeline & GridSearch

This notebook covers:
- Phase 9: Pipeline & GridSearch for Hyperparameter Optimization

**Note:** Run `3_modelling.ipynb` first to identify the best model.


In [1]:
# Import libraries
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor

# Set random seed for reproducibility
RANDOM_STATE = 777
np.random.seed(RANDOM_STATE)

print("Libraries imported successfully!")


Libraries imported successfully!


In [2]:
# Load preprocessed data and final model info from modelling.ipynb
with open('models/preprocessor.pkl', 'rb') as f:
    preprocessor = pickle.load(f)

# Reload and prepare data (same as modelling.ipynb)
df = pd.read_csv('data/spotify-tracks.csv')
columns_to_drop = ['spotify_id', 'name', 'artists', 'album_name', 'album_release_date',
                   'popular_in_country', 'mode', 'is_explicit', 'release_year', 
                   'key', 'time_signature', 'release_month', 'duration_ms', 'popularity']
df_clean = df.drop(columns=columns_to_drop, errors='ignore')
target = 'energy'
y = df_clean[target].copy()
X = df_clean.drop(columns=[target]).copy()

# Feature engineering
X_engineered = X.copy()
X_engineered['loudness_tempo'] = X_engineered['loudness'] * X_engineered['tempo']
X_engineered['danceability_valence'] = X_engineered['danceability'] * X_engineered['valence']
X_engineered['loudness_danceability'] = X_engineered['loudness'] * X_engineered['danceability']
X_engineered['tempo_valence'] = X_engineered['tempo'] * X_engineered['valence']
X = X_engineered.copy()

# Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)
feature_names = preprocessor.get_feature_names_out()

# Get top 6 features from modelling.ipynb (you'll need to run that first)
print(f"Data loaded: Train={X_train_processed.shape}, Test={X_test_processed.shape}")


Data loaded: Train=(17268, 14), Test=(4317, 14)


## Phase 9: Hyperparametertuning via GridSearch on the top two Models


In [3]:
# Feature selector transformer
class TopFeatureSelector(BaseEstimator, TransformerMixin):
    """Selects top N features"""
    def __init__(self, feature_indices):
        self.feature_indices = feature_indices
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[:, self.feature_indices]

# Load top 6 indices from modelling.ipynb
try:
    with open('models/top_6_indices.pkl', 'rb') as f:
        top_6_indices = pickle.load(f)
    print(f"✅ Loaded top_6_indices from modelling.ipynb: {top_6_indices}")
except FileNotFoundError:
    print("⚠️ top_6_indices.pkl not found. Run modelling.ipynb first!")
    print("   Using first 6 features as fallback...")
    top_6_indices = list(range(6))

✅ Loaded top_6_indices from modelling.ipynb: [3, 9, 4, 12, 13, 5]


In [4]:
# Create pipeline with feature selector and model
# Example: Optimizing Gradient Boosting
pipeline = Pipeline([
    ('feature_selector', TopFeatureSelector(top_6_indices)),
    ('model', GradientBoostingRegressor(random_state=RANDOM_STATE))
])

# Define parameter grid
param_grid = {
    'model__n_estimators': [100, 200, 300],
    'model__max_depth': [3, 5, 7],
    'model__learning_rate': [0.01, 0.1, 0.2],
    'model__subsample': [0.8, 0.9, 1.0]
}

print("=" * 60)
print("PIPELINE & GRID SEARCH (GRADIENT BOOSTING REGRESSOR)")
print("=" * 60)

print(f"\nParameter grid:")
for param, values in param_grid.items():
    print(f"  {param}: {values}")

# GridSearch
print("\n" + "=" * 60)
print("RUNNING GRID SEARCH")
print("=" * 60)
print("This may take a while... (with our Hardware it took ~6 minutes)")

grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train_processed, y_train)


PIPELINE & GRID SEARCH (GRADIENT BOOSTING REGRESSOR)

Parameter grid:
  model__n_estimators: [100, 200, 300]
  model__max_depth: [3, 5, 7]
  model__learning_rate: [0.01, 0.1, 0.2]
  model__subsample: [0.8, 0.9, 1.0]

RUNNING GRID SEARCH
This may take a while... (with our Hardware it took ~6 minutes)
Fitting 5 folds for each of 81 candidates, totalling 405 fits


,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step..._state=777))])
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'model__learning_rate': [0.01, 0.1, ...], 'model__max_depth': [3, 5, ...], 'model__n_estimators': [100, 200, ...], 'model__subsample': [0.8, 0.9, ...]}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'r2'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",5
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation tim

In [5]:
# Results
print("\n" + "=" * 60)
print("GRID SEARCH RESULTS (GRADIENT BOOSTING REGRESSOR)")
print("=" * 60)
print(f"Best R² (CV): {grid_search.best_score_:.4f}")
print(f"Best parameters:")
for param, value in grid_search.best_params_.items():
    print(f"  {param}: {value}")

# Evaluate on test set
from sklearn.metrics import r2_score, mean_squared_error
y_pred_optimized = grid_search.predict(X_test_processed)
r2_optimized_gradient = r2_score(y_test, y_pred_optimized)
rmse_optimized = np.sqrt(mean_squared_error(y_test, y_pred_optimized))

print(f"\nTest Set Performance:")
print(f"  R²: {r2_optimized_gradient:.4f}")


GRID SEARCH RESULTS (GRADIENT BOOSTING REGRESSOR)
Best R² (CV): 0.6785
Best parameters:
  model__learning_rate: 0.1
  model__max_depth: 7
  model__n_estimators: 100
  model__subsample: 0.8

Test Set Performance:
  R²: 0.6967


In [6]:
print("=" * 60)
print("PIPELINE & GRID SEARCH (RANDOM FOREST)")
print("=" * 60)

# ------------------------------------------------
# Create pipeline with feature selector + model
# ------------------------------------------------
pipeline_rf = Pipeline([
    ('feature_selector', TopFeatureSelector(top_6_indices)),
    ('model', RandomForestRegressor(
        random_state=RANDOM_STATE,
        n_jobs=-1
    ))
])

# ------------------------------------------------
# Define parameter grid
# ------------------------------------------------
param_grid_rf = {
    'model__n_estimators': [100, 300, 600],
    'model__max_depth': [None, 10, 20],
    'model__min_samples_split': [2, 5, 10],
    'model__min_samples_leaf': [1, 2, 4],
    'model__max_features': ['sqrt', 'log2']
}

print("\nParameter grid:")
for param, values in param_grid_rf.items():
    print(f"  {param}: {values}")

# ------------------------------------------------
# Grid Search
# ------------------------------------------------
print("\n" + "=" * 60)
print("RUNNING GRID SEARCH (RANDOM FOREST)")
print("=" * 60)
print("This may take a while... (with our Hardware it took ~14 minutes)")

grid_search_rf = GridSearchCV(
    pipeline_rf,
    param_grid_rf,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    verbose=1
)

grid_search_rf.fit(X_train_processed, y_train)

PIPELINE & GRID SEARCH (RANDOM FOREST)

Parameter grid:
  model__n_estimators: [100, 300, 600]
  model__max_depth: [None, 10, 20]
  model__min_samples_split: [2, 5, 10]
  model__min_samples_leaf: [1, 2, 4]
  model__max_features: ['sqrt', 'log2']

RUNNING GRID SEARCH (RANDOM FOREST)
This may take a while... (with our Hardware it took ~14 minutes)
Fitting 5 folds for each of 162 candidates, totalling 810 fits


/Users/endrass/Documents/70_Masterstudium/3_Semester/Predictive-Analytics/homework_spotify-algorithmus/.venv/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/endrass/Documents/70_Masterstudium/3_Semester/Predictive-Analytics/homework_spotify-algorithmus/.venv/lib/python3.12/site-packages/sklearn/utils/parallel.py:144: UserWarning: `sklearn.utils.parallel.delayed` should be used with `sklearn.utils.parallel.Parallel` to make it possible to propagate the scikit-learn configuration of the current thread to the joblib workers.
  warnings.warn(
/Users/endrass/Documents/70_Masterstudium/3_Semester/Predictive-Analytics/homework_spotify-algorithmus/.venv/lib/python3.12/site-packages/sklearn/utils/parallel.py:144: UserWarning: `sklearn.utils.parallel.delayed` should be used with `sklearn.utils.parallel.

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step..._state=777))])
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'model__max_depth': [None, 10, ...], 'model__max_features': ['sqrt', 'log2'], 'model__min_samples_leaf': [1, 2, ...], 'model__min_samples_split': [2, 5, ...], ...}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'r2'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",5
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the com

In [7]:
# ------------------------------------------------
# Results
# ------------------------------------------------
print("\n" + "=" * 60)
print("GRID SEARCH RESULTS (RANDOM FOREST)")
print("=" * 60)

print(f"Best R² (CV): {grid_search_rf.best_score_:.4f}")
print("Best parameters:")
for param, value in grid_search_rf.best_params_.items():
    print(f"  {param}: {value}")

# ------------------------------------------------
# Evaluate on test set
# ------------------------------------------------
y_pred_optimized = grid_search_rf.predict(X_test_processed)
r2_optimized_forest = r2_score(y_test, y_pred_optimized)
rmse_optimized = np.sqrt(mean_squared_error(y_test, y_pred_optimized))

print(f"\nTest Set Performance:")
print(f"  R²: {r2_optimized_forest:.4f}")
print(f"  RMSE: {rmse_optimized:.4f}")



GRID SEARCH RESULTS (RANDOM FOREST)
Best R² (CV): 0.6968
Best parameters:
  model__max_depth: None
  model__max_features: sqrt
  model__min_samples_leaf: 1
  model__min_samples_split: 2
  model__n_estimators: 600

Test Set Performance:
  R²: 0.7131
  RMSE: 0.0926


In [8]:
print("\n" + "=" * 60)
print("COMPARISON OF TUNED MODEL OPTIONS")
print("=" * 60)

print(f"{'Model':<35} {'R²':>8}")
print("-" * 60)

print(f"{'Gradient Boosting (Tuned)':<35} {r2_optimized_gradient:>8.4f}")
print(f"{'Random Forest (Tuned, Top 6)':<35} {r2_optimized_forest:>8.4f}")

print("-" * 60)

best_model = max(
    [
        ('Gradient Boosting (Tuned)', r2_optimized_gradient),
        ('Random Forest (Tuned, Top 6)', r2_optimized_forest)
    ],
    key=lambda x: x[1]
)

print(f"\n🏆 Selected Tuned Model: {best_model[0]}")
print("used Parameters:")
for param, value in grid_search_rf.best_params_.items():
    print(f"  {param}: {value}")
print("=" * 60)



COMPARISON OF TUNED MODEL OPTIONS
Model                                     R²
------------------------------------------------------------
Gradient Boosting (Tuned)             0.6967
Random Forest (Tuned, Top 6)          0.7131
------------------------------------------------------------

🏆 Selected Tuned Model: Random Forest (Tuned, Top 6)
used Parameters:
  model__max_depth: None
  model__max_features: sqrt
  model__min_samples_leaf: 1
  model__min_samples_split: 2
  model__n_estimators: 600


## Summary

In this notebook we:
1. ✅ Created a pipeline with feature selector and model
2. ✅ Defined parameter grid for hyperparameter tuning
3. ✅ Ran GridSearchCV with 5-fold cross-validation
4. ✅ Found best hyperparameters
5. ✅ Evaluated optimized model on test set

**Next:** Move to `5_train_final_model.ipynb` to save the final model and create usage examples.
